# Experimentation and uplift testing

## Load data

In [11]:
import pandas as pd

In [12]:
cust_transact_data = pd.read_csv("./data/QVI_data.csv")
cust_transact_data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [13]:
cust_transact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


## Changing to correct types

Converting `DATE` column to date type.

In [14]:
cust_transact_data["DATE"] = pd.to_datetime(cust_transact_data["DATE"])

Converting `PROD_NAME`, `BRAND`, `LIFESTAGE`, and `PREMIUM_CUSTOMER` to category type.

In [15]:
cust_transact_data["PROD_NAME"] = cust_transact_data["PROD_NAME"].astype("category")
cust_transact_data["BRAND"] = cust_transact_data["BRAND"].astype("category")
cust_transact_data["LIFESTAGE"] = cust_transact_data["LIFESTAGE"].astype("category")
cust_transact_data["PREMIUM_CUSTOMER"] = cust_transact_data["PREMIUM_CUSTOMER"].astype("category")

In [16]:
cust_transact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int64         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int64         
 3   TXN_ID            264834 non-null  int64         
 4   PROD_NBR          264834 non-null  int64         
 5   PROD_NAME         264834 non-null  category      
 6   PROD_QTY          264834 non-null  int64         
 7   TOT_SALES         264834 non-null  float64       
 8   PACK_SIZE         264834 non-null  int64         
 9   BRAND             264834 non-null  category      
 10  LIFESTAGE         264834 non-null  category      
 11  PREMIUM_CUSTOMER  264834 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1), int64(6)
memory usage: 17.2 MB


## Removing stores which don't have 12 months data

In [17]:
cust_transact_data["MONTH"] = cust_transact_data["DATE"].dt.month
cust_transact_data["MONTH"]

0         10
1          9
2          3
3          3
4         11
          ..
264829    12
264830    10
264831    10
264832    10
264833    12
Name: MONTH, Length: 264834, dtype: int32

In [18]:
data_by_date = cust_transact_data.groupby("STORE_NBR")["MONTH"].nunique()

In [19]:
data_by_date

STORE_NBR
1      12
2      12
3      12
4      12
5      12
       ..
268    12
269    12
270    12
271    12
272    12
Name: MONTH, Length: 272, dtype: int64

In [20]:
stores_to_remove = data_by_date[data_by_date != 12].index

In [21]:
cust_transact_data = cust_transact_data[~cust_transact_data["STORE_NBR"].isin(stores_to_remove)]

## Splitting the data based on pre trial and trial period

### Pretrial data

In [22]:
cust_transact_data_pretrial = cust_transact_data[cust_transact_data["DATE"] < "2019-02-01"]

In [23]:
cust_transact_data_pretrial.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,MONTH
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,9
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,11
5,1005,2018-12-28,1,6,86,Cheetos Puffs 165g,1,2.8,165,CHEETOS,MIDAGE SINGLES/COUPLES,Mainstream,12
6,1007,2018-12-04,1,7,49,Infuzions SourCream&Herbs Veg Strws 110g,1,3.8,110,INFUZIONS,YOUNG SINGLES/COUPLES,Budget,12


In [24]:
print("Min date: ", cust_transact_data_pretrial["DATE"].min())
print("Max date: ", cust_transact_data_pretrial["DATE"].max())

Min date:  2018-07-01 00:00:00
Max date:  2019-01-31 00:00:00


### Trial data

The trial date is between start of february to end of april.

In [25]:
cust_transact_data_trial = cust_transact_data[
    (cust_transact_data["DATE"] > "2019-02-01") &
    (cust_transact_data["DATE"] < "2019-05-01")
]

In [26]:
print("Min date: ", cust_transact_data_trial["DATE"].min())
print("Max date: ", cust_transact_data_trial["DATE"].max())

Min date:  2019-02-02 00:00:00
Max date:  2019-04-30 00:00:00


## Metrics for comparison

The following metrics will be used to select control stores similar to trial stores
* monthly total sales revenue
* monthly total number of customers
* monthly average number of transactions per customer

In [30]:
tot_sales = cust_transact_data_pretrial.groupby(["STORE_NBR","MONTH"])["TOT_SALES"].sum()

total_num_cust = (cust_transact_data_pretrial
                 .groupby(["STORE_NBR","MONTH"])["LYLTY_CARD_NBR"].nunique()
                 .rename("NUM_OF_CUST"))

avg_cust_transact = (cust_transact_data_pretrial
               .groupby(["STORE_NBR","MONTH"])["TOT_SALES"].mean()
               .rename("AVG_TRANSACT"))


cust_transact_summary = (pd.concat([tot_sales, total_num_cust, avg_cust_transact], axis=1))
                         

cust_transact_summary

TOT_SALES  NUM_OF_CUST  AVG_TRANSACT
STORE_NBR MONTH                                      
1         1         154.80           35      4.300000
          7         206.90           49      3.978846
          8         176.10           42      4.095349
          9         278.80           59      4.496774
          10        188.10           44      4.180000
...                    ...          ...           ...
272       8         372.85           44      7.767708
          9         304.70           32      8.463889
          10        430.60           44      8.443137
          11        376.20           41      8.360000
          12        403.90           47      8.593617

[1820 rows x 3 columns]

In [34]:
cust_transact_summary.loc[1]

,TOT_SALES,NUM_OF_CUST,AVG_TRANSACT
MONTH,,,
1,154.8,35,4.300000
7,206.9,49,3.978846
8,176.1,42,4.095349
9,278.8,59,4.496774
10,188.1,44,4.180000
11,192.6,46,4.097872
12,189.6,42,4.034043


## Function to get control store

In [63]:
def get_control_store(data, trial_store):
    data = data.copy()
    data = data.sort_values(["STORE_NBR", "MONTH"], ascending=False)
    
    corr_scores = pd.DataFrame(columns=["STORE_NBR", "TOT_SALES", "NUM_OF_CUST", "AVG_TRANSACT"])
    
    stores = cust_transact_summary.index.get_level_values(0).unique().to_list()
    
    for i in stores:
        store = pd.Series([i])
        store.index = ["STORE_NBR"]
        corr_score_i = pd.concat(
            [store,
            cust_transact_summary.loc[1].corrwith(cust_transact_summary.loc[2])]
        )
        corr_scores = pd.concat(
            [corr_scores,
             pd.DataFrame([corr_score_i.to_list()], columns=corr_score_i.index)],
            ignore_index=True
        )
        
        
    return corr_scores
    

In [35]:
cust_transact_summary.loc[1].corrwith(cust_transact_summary.loc[2])

TOT_SALES      -0.308766
NUM_OF_CUST    -0.588159
AVG_TRANSACT    0.223059
dtype: float64

In [ ]:
cust_transact_summary.index.get_level_values(0).unique().to_list()

In [64]:
corr = get_control_store(cust_transact_summary, 77)

In [65]:
corr

,STORE_NBR,TOT_SALES,NUM_OF_CUST,AVG_TRANSACT
0,1.0,-0.308766,-0.588159,0.223059
1,2.0,-0.308766,-0.588159,0.223059
2,3.0,-0.308766,-0.588159,0.223059
3,4.0,-0.308766,-0.588159,0.223059
4,5.0,-0.308766,-0.588159,0.223059
...,...,...,...,...
255,268.0,-0.308766,-0.588159,0.223059
256,269.0,-0.308766,-0.588159,0.223059
257,270.0,-0.308766,-0.588159,0.223059
258,271.0,-0.308766,-0.588159,0.223059
